# Step-Back Prompting (Question-Answering)

One prompting technique called "Step-Back" prompting can improve performance on complex questions by first asking a "step back" question. This can be combined with regular question-answering applications by then doing retrieval on both the original and step-back question.

Read the paper [here](https://arxiv.org/abs/2310.06117)

See an excellent blog post on this by Cobus Greyling [here](https://cobusgreyling.medium.com/a-new-prompt-engineering-technique-has-been-introduced-called-step-back-prompting-b00e8954cacb)

In this cookbook we will replicate this technique. We modify the prompts used slightly to work better with chat models.

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda

In [24]:
# Few Shot Examples
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?"
    },
    {
        "input": "Jan Sindel’s was born in what country?", 
        "output": "what is Jan Sindel’s personal history?"
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [25]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:"""),
    # Few shot examples
    few_shot_prompt,
    # New question
    ("user", "{question}"),
])

In [26]:
question_gen = prompt | ChatOpenAI(temperature=0) | StrOutputParser()

In [27]:
question = "was chatgpt around while trump was president?"

In [28]:
question_gen.invoke({"question": question})

"what is the timeline of ChatGPT's existence?"

In [29]:
from langchain.utilities import DuckDuckGoSearchAPIWrapper


search = DuckDuckGoSearchAPIWrapper(max_results=4)

def retriever(query):
    return search.run(query)

In [30]:
retriever(question)

'While impressive in many respects, ChatGPT also has some major flaws. ... [President\'s Name]," refused to write a poem about ex-President Trump, but wrote one about President Biden ... On Wednesday, a Twitter user posted screenshots of him asking OpenAI\'s chatbot, ChatGPT, to write a positive poem about former President Donald Trump, to which the chatbot declined, citing it ... This includes content about former President Donald Trump. According to further tests, ChatGPT successfully wrote poems admiring all recent U.S. presidents, but failed when we entered a query for ... During the Trump administration, Altman gained new attention as a vocal critic of the president. It was against that backdrop that he was rumored to be considering a run for California governor.'

In [31]:
retriever(question_gen.invoke({"question": question}))

"Will Douglas Heaven March 3, 2023 Stephanie Arnett/MITTR | Envato When OpenAI launched ChatGPT, with zero fanfare, in late November 2022, the San Francisco-based artificial-intelligence company... ChatGPT, which stands for Chat Generative Pre-trained Transformer, is a large language model -based chatbot developed by OpenAI and launched on November 30, 2022, which enables users to refine and steer a conversation towards a desired length, format, style, level of detail, and language. ChatGPT is an artificial intelligence (AI) chatbot built on top of OpenAI's foundational large language models (LLMs) like GPT-4 and its predecessors. This chatbot has redefined the standards of... The transformer architecture behind today's generation of LLMs was introduced in 2017 by a team of Google researchers."

In [32]:
# response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
# response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

In [33]:
from langchain import hub

response_prompt = hub.pull("langchain-ai/stepback-answer")

In [34]:
chain = {
    # Retrieve context using the normal question
    "normal_context": RunnableLambda(lambda x: x['question']) | retriever,
    # Retrieve context using the step-back question
    "step_back_context": question_gen | retriever,
    # Pass on the question
    "question": lambda x: x["question"]
} | response_prompt | ChatOpenAI(temperature=0) | StrOutputParser()

In [35]:
chain.invoke({"question": question})

"Based on the given context, it is not explicitly mentioned whether ChatGPT was around while Donald Trump was president. The information provided only states that a Twitter user asked ChatGPT to write a positive poem about former President Donald Trump, and the chatbot declined the request. However, it does not specify the timeline of this interaction or whether ChatGPT existed during Trump's presidency.\n\nTo provide a comprehensive answer, we need to rely on general knowledge. ChatGPT, developed by OpenAI, was introduced to the world at the end of 2022. It is a language model that uses artificial intelligence to generate human-like text responses. Therefore, it is reasonable to assume that ChatGPT was not around during the entire duration of Donald Trump's presidency, as it was introduced after his term ended.\n\nHowever, it is important to note that the given context does not provide a direct confirmation or denial of ChatGPT's existence during Trump's presidency."

## Baseline

In [36]:
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

{normal_context}

Original Question: {question}
Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

In [37]:
chain = {
    # Retrieve context using the normal question (only the first 3 results)
    "normal_context": RunnableLambda(lambda x: x['question']) | retriever,
    # Pass on the question
    "question": lambda x: x["question"]
} | response_prompt | ChatOpenAI(temperature=0) | StrOutputParser()

In [38]:
chain.invoke({"question": question})

"Yes, ChatGPT was around while Donald Trump was president. However, it is important to note that ChatGPT's responses are generated based on pre-existing data and training, and it does not have real-time knowledge or awareness of specific events or timelines. Therefore, its response to a request for a poem about former President Trump may have been influenced by the data it was trained on, which could include biases or limitations in its understanding of current events."